In [3]:
import pandas as pd
import numpy as np

raw_train = pd.read_csv(filepath_or_buffer="~/.kaggle/competitions/titanic/train.csv")
raw_test = pd.read_csv(filepath_or_buffer="~/.kaggle/competitions/titanic/test.csv")

raw_train["is_test"] = False
raw_test["is_test"] = True

all_data = pd.concat((raw_train, raw_test))

In [4]:
all_data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,is_test
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,False
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,False
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,False
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,False
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,False


In [40]:
# Refer to: [Titanic with Keras | Kaggle](https://www.kaggle.com/cstahl12/titanic-with-keras)

def get_title_last_name(name):
    full_name = name.str.split(', ', n=0, expand=True)
    # last_name = full_name[0] # last_name は使っていない模様
    titles = full_name[1].str.split('.', n=0, expand=True)
    titles = titles[0]
    return(titles)

def get_titles_from_names(df):
    """
    """
    df['Title'] = get_title_last_name(df['Name'])
    df = df.drop(['Name'], axis=1)
    return(df)

def get_cabin_letter(df):
    df['Cabin'].fillna('Z', inplace=True) # Cabin が空欄の行は "Z" で埋める
    df['Cabin_letter'] = df['Cabin'].str[0] # ?例えば "C91" ならば "C" でまとめてしまっても問題ないということ？なぜ？
    return(df)

def get_dummy_cats(df):
    return(pd.get_dummies(df, columns=['Title', 'Pclass', 'Sex', 'Embarked',
                                       'Cabin', 'Cabin_letter'])) # なんだこの便利メソッドは！

def process_data(df):
    # preprocess titles, cabin, embarked
    df = get_titles_from_names(df)
    df['Embarked'].fillna('S', inplace=True) # よくよくデータを見ると、Embarkedが空欄の行がある
    df = get_cabin_letter(df)
    
    # drop remaining features
    df = df.drop(['Ticket', 'Fare'], axis=1) # ?なぜdropする？
    
    # create dummies for categorial features
    df = get_dummy_cats(df)
    
    return(df)

proc_data = process_data(all_data)

proc_train = proc_data[proc_data["is_test"] == False]
proc_eval = proc_data[proc_data["is_test"] == False]
proc_test = proc_data[proc_data["is_test"] == True]

Build Network to predict missing ages

In [41]:
for_age_train = proc_data.drop(['Survived', 'is_test'], axis=1).dropna(axis=0)
X_train_age = for_age_train.drop('Age', axis=1)
y_train_age = for_age_train['Age']

Create a model to predict missing age

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

tmodel = Sequential()
tmodel.add(Dense(input_dim=X_train_age.shape[1], units=128,
                 kernel_initializer='normal', bias_initializer='zeros'))
tmodel.add(Activation('relu'))

for i in range(0, 8):
    tmodel.add(Dense(units=64, kernel_initializer='normal',
                     bias_initializer='zeros'))
    tmodel.add(Activation('relu'))
    tmodel.add(Dropout(.25))

tmodel.add(Dense(units=1))
tmodel.add(Activation('linear'))

tmodel.compile(loss='mean_squared_error', optimizer='rmsprop')

Fit the model

In [43]:
tmodel.fit(X_train_age.values, y_train_age.values, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 638.5478
Epoch 2/100
 - 0s - loss: 490.8742
Epoch 3/100
 - 0s - loss: 472.5989
Epoch 4/100
 - 0s - loss: 481.8364
Epoch 5/100
 - 0s - loss: 443.9114
Epoch 6/100
 - 0s - loss: 427.7031
Epoch 7/100
 - 0s - loss: 382.8816
Epoch 8/100
 - 0s - loss: 355.4669
Epoch 9/100
 - 0s - loss: 300.2643
Epoch 10/100
 - 0s - loss: 249.0685
Epoch 11/100
 - 0s - loss: 251.8090
Epoch 12/100
 - 0s - loss: 241.1803
Epoch 13/100
 - 0s - loss: 233.5047
Epoch 14/100
 - 0s - loss: 239.7554
Epoch 15/100
 - 0s - loss: 236.8429
Epoch 16/100
 - 0s - loss: 237.3041
Epoch 17/100
 - 0s - loss: 239.0178
Epoch 18/100
 - 0s - loss: 234.2993
Epoch 19/100
 - 0s - loss: 234.9820
Epoch 20/100
 - 0s - loss: 272.3264
Epoch 21/100
 - 0s - loss: 224.4681
Epoch 22/100
 - 0s - loss: 225.8648
Epoch 23/100
 - 0s - loss: 226.2237
Epoch 24/100
 - 0s - loss: 222.9897
Epoch 25/100
 - 0s - loss: 225.3766
Epoch 26/100
 - 0s - loss: 232.1095
Epoch 27/100
 - 0s - loss: 214.8952
Epoch 28/100
 - 0s - loss: 200.1617
E

In [44]:
to_pred = proc_train.loc[proc_train['Age'].isnull()].drop(
          ['Age', 'Survived', 'is_test'], axis=1)
p = tmodel.predict(to_pred.values)

proc_train['Age'] = proc_train['Age'].fillna(pd.Series(p[:,0]))

/Users/yahata/.pyenv/versions/3.6.3/envs/venv-3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Build network to predict "Survived"

In [19]:
X = proc_train.drop(["Survived", "is_test"], axis=1)
y = pd.get_dummies(proc_train["Survived"]) # ?なぜこうする？

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# create model
model = Sequential()
model.add(Dense(input_dim=X.shape[1], units=128,
                 kernel_initializer='normal', bias_initializer='zeros'))
model.add(Activation('relu'))

for i in range(0, 15):
    model.add(Dense(units=128, kernel_initializer='normal',
                     bias_initializer='zeros'))
    model.add(Activation('relu'))
    model.add(Dropout(.40))

model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X.values, y.values,
          validation_split=0.1,
          epochs=20,
          batch_size=128)

Train on 801 samples, validate on 90 samples
Epoch 1/20
801/801 [==============================] - 2s 2ms/step - loss: 0.6893 - acc: 0.5868 - val_loss: 0.6816 - val_acc: 0.6222
Epoch 2/20
801/801 [==============================] - 0s 143us/step - loss: 0.6775 - acc: 0.6155 - val_loss: 0.6662 - val_acc: 0.6222
Epoch 3/20
801/801 [==============================] - 0s 155us/step - loss: 0.6713 - acc: 0.6155 - val_loss: 0.6639 - val_acc: 0.6222
Epoch 4/20
801/801 [==============================] - 0s 152us/step - loss: 0.6688 - acc: 0.6155 - val_loss: 0.6629 - val_acc: 0.6222
Epoch 5/20
801/801 [==============================] - 0s 151us/step - loss: 0.6673 - acc: 0.6155 - val_loss: 0.6630 - val_acc: 0.6222
Epoch 6/20
801/801 [==============================] - 0s 148us/step - loss: 0.6700 - acc: 0.6155 - val_loss: 0.6636 - val_acc: 0.6222
Epoch 7/20
801/801 [==============================] - 0s 151us/step - loss: 0.6672 - acc: 0.6155 - val_loss: 0.6650 - val_acc: 0.6222
Epoch 8/20
801/801 

In [13]:
X_test = proc_test.drop(["Survived", "is_test"], axis=1)

p_survived = model.predict(X_test.values, batch_size=128)